# Backtest External Models

We continue to illustrate how to use `Backtest` object to gauge performance of external models. `Backtest` is designed to work for any model objects which have a `fit` method and a `predict` method, by writing some call back functions. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

from orbit.backtest.backtest import TimeSeriesSplitter, Backtest
from orbit.utils.metrics import mape, smape, wmape

ModuleNotFoundError: No module named 'orbit.backtest.backtest'

## Load data

In [ ]:
data_path = "../examples/data/iclaims_example.csv"
raw_data = pd.read_csv(data_path, parse_dates=['week'])

## log transformation
data = raw_data.copy()
# data[['claims', 'trend.unemploy', 'trend.filling', 'trend.job']] = \
#     data[['claims', 'trend.unemploy', 'trend.filling', 'trend.job']].apply(np.log, axis=1)

print(data.shape)
data.head(5)

## Sklearn model object - Random Forest

Declare a `TimeSeriesSplitter` and `Backtest` object.

In [ ]:
splitter = TimeSeriesSplitter(data, min_train_len=200, incremental_len=20, forecast_len=20, n_splits=3)

In [ ]:
bt = Backtest(splitter=splitter)

Next, we instantiate an sklearn model.

In [ ]:
from sklearn.ensemble import RandomForestRegressor

mod = RandomForestRegressor(n_estimators = 50)

Create callback functions for the `RandomForestRegressor` model.

In [ ]:
def fit_callback_sklearn(model, train_df, response_col, regressor_col):
    y = train_df[response_col]
    X = train_df[regressor_col]
    model.fit(X, y)
    return

def predict_callback_sklearn(model, test_df, response_col, regressor_col):
    X = test_df[regressor_col]
    pred = model.predict(X)

    return pd.DataFrame(pred, columns=['prediction'])

# passed into fit_callback_sklearn()
fit_predict_args = {
    'response_col': 'claims',
    'regressor_col': ['trend.unemploy', 'trend.filling', 'trend.job']
}

In [ ]:
bt.fit_score(
    mod,
    response_col='claims',
    predicted_col='prediction',
    fit_callback=fit_callback_sklearn,
    predict_callback=predict_callback_sklearn,
    fit_args=fit_predict_args,
    predict_args=fit_predict_args
)

bt.get_predictions().head(5)

In [ ]:
bt.get_scores()

## Prophet model object

In [ ]:
from fbprophet import Prophet
import inspect

In [ ]:
def model_callback_prophet(model, **kwargs):
    object_type = type(model)
    new_instance = object_type(**kwargs)
    
    return new_instance

def fit_callbacks_prophet(model, train_df, date_col, response_col, regressor_col):
    
    train_df = train_df.rename(columns={date_col: "ds", response_col: "y"})
    if regressor_col is not None:
        for regressor in regressor_col:
            model.add_regressor(regressor) 
    model.fit(train_df)
    
    return 

def pred_callbacks_prophet(model, test_df, date_col, response_col, regressor_col):
    test_df = test_df.rename(columns={date_col: "ds", response_col: "y"})
    
    predictions = model.predict(test_df)
    predictions.rename(columns={'yhat': 'prediction', 'ds': date_col}, inplace=True)
    predictions=predictions[[date_col, 'prediction']]

    return predictions

fit_predict_args = {
    'response_col': 'claims',
    'date_col': 'week',
    'regressor_col': ['trend.unemploy', 'trend.filling', 'trend.job']
}

In [ ]:
mod = Prophet()

Note: if some error pops out below (related to the pickle issue), it could be eliminated by setting `save_model=False` or upgrading your python to 3.7.

In [ ]:
bt.fit_score(
    mod,
    response_col='claims',
    predicted_col='prediction',
    fit_callback=fit_callbacks_prophet,
    predict_callback=pred_callbacks_prophet,
    model_callback=model_callback_prophet,
    fit_args=fit_predict_args,
    predict_args=fit_predict_args,
)

In [ ]:
bt.get_predictions().head()

In [ ]:
bt.get_scores()